# Reshaping DataFrames Using Pandas




## Outline
* Wide to Long with `melt`
* Long to Wide with `pivot`




In [13]:
import pandas as pd
from pathlib import Path

# our dataset:
df = pd.read_csv(Path('data/mods_data_wide.csv'))
df

,id,status,data_worker,Manager,pursing_work,not_in_field
0,1,Completed,1.0,NaN,NaN,NaN
1,2,Completed,1.0,NaN,NaN,NaN
2,3,Completed,1.0,NaN,NaN,NaN
3,4,Completed,1.0,NaN,NaN,NaN
4,5,Completed,1.0,NaN,NaN,NaN
...,...,...,...,...,...,...
329,330,Completed,1.0,NaN,NaN,NaN
330,331,Completed,NaN,NaN,1.0,NaN
331,332,Completed,NaN,NaN,1.0,NaN
332,333,Completed,1.0,NaN,NaN,NaN


In [36]:
df.csv_out(mods.csv)

NameError: name 'df' is not defined

# Wide to Long with `melt`

Our data currently has a different column for each variable, or "wide format".  Use `DataFrame.melt()` to convert it to long form, where the resulting table will have a "variable" column containing the variable name, and a "value" column containing the value of that variable.

We can use this functionality to convert our "data_worker", "Manager", "pursuing_work", and "not_in_field" columns into a column called "work_type", whose value is one of the 4 types of work.

Pick the identifying columns for the `id_vars` argument, and include all variables you want unpivoted in `value_vars`.

In [14]:
df.melt(
    id_vars='id', 
    value_vars=['data_worker', 'Manager'])

,id,variable,value
0,1,data_worker,1.0
1,2,data_worker,1.0
2,3,data_worker,1.0
3,4,data_worker,1.0
4,5,data_worker,1.0
...,...,...,...
663,330,Manager,NaN
664,331,Manager,NaN
665,332,Manager,NaN
666,333,Manager,NaN


`melt()` will also assume all columns are `value_vars` if they aren't included as `id_vars`. Use `var_name` and `value_name` to control the names of the output dataframe columns.

In [4]:
melted = df.melt(
    id_vars=['id', 'status'], 
    var_name='work_type', 
    value_name='has_data')

melted

,id,status,work_type,has_data
0,1,Completed,data_worker,1.0
1,2,Completed,data_worker,1.0
2,3,Completed,data_worker,1.0
3,4,Completed,data_worker,1.0
4,5,Completed,data_worker,1.0
...,...,...,...,...
1331,330,Completed,not_in_field,NaN
1332,331,Completed,not_in_field,NaN
1333,332,Completed,not_in_field,NaN
1334,333,Completed,not_in_field,NaN


Predictably, the resulting dataframe has a row count equal to the original number of rows multiplied by the number of features included as `value_vars`.

In [5]:
if len(melted) == len(df) * 4 == 1336: # there are 5 feature variables that we are melting
    print('okay!')
else:
    print('ERROR')

okay!


We now have a column called work_type that has the types of work in it, however we also have duplicate rows where the 'has_data' field is empty (or NaN).  We only want the rows where the value is 1.  Remember selecting data from a dataframe?

In [150]:
melted = melted[melted.has_data == 1]

melted

,id,status,work_type,has_data
0,1,Completed,data_worker,1.0
1,2,Completed,data_worker,1.0
2,3,Completed,data_worker,1.0
3,4,Completed,data_worker,1.0
4,5,Completed,data_worker,1.0
...,...,...,...,...
1228,227,Completed,not_in_field,1.0
1236,235,Completed,not_in_field,1.0
1275,274,Completed,not_in_field,1.0
1276,275,Completed,not_in_field,1.0


Our 'has_data' colunn is now all 1's, so we might as well drop it.

In [151]:
melted = melted.drop(columns='has_data')

In [152]:
df # original dataframe

,id,status,data_worker,Manager,pursing_work,not_in_field
0,1,Completed,1.0,NaN,NaN,NaN
1,2,Completed,1.0,NaN,NaN,NaN
2,3,Completed,1.0,NaN,NaN,NaN
3,4,Completed,1.0,NaN,NaN,NaN
4,5,Completed,1.0,NaN,NaN,NaN
...,...,...,...,...,...,...
329,330,Completed,1.0,NaN,NaN,NaN
330,331,Completed,NaN,NaN,1.0,NaN
331,332,Completed,NaN,NaN,1.0,NaN
332,333,Completed,1.0,NaN,NaN,NaN


In [153]:
melted # finished conversion

,id,status,work_type
0,1,Completed,data_worker
1,2,Completed,data_worker
2,3,Completed,data_worker
3,4,Completed,data_worker
4,5,Completed,data_worker
...,...,...,...
1228,227,Completed,not_in_field
1236,235,Completed,not_in_field
1275,274,Completed,not_in_field
1276,275,Completed,not_in_field


Let's look at some data to check our work:

In [12]:
def check(id):
    orig_row = df[df.id == id]
    melted_row = melted[melted.id == id]
    print(orig_row)
    print(' ')
    print(melted_row)
    print('\n')

check(129)
check(301)

      id     status  data_worker  Manager  pursing_work  not_in_field
128  129  Completed          NaN      1.0           NaN           NaN
 
       id     status     work_type  has_data
128   129  Completed   data_worker       NaN
462   129  Completed       Manager       1.0
796   129  Completed  pursing_work       NaN
1130  129  Completed  not_in_field       NaN


      id     status  data_worker  Manager  pursing_work  not_in_field
300  301  Completed          1.0      NaN           NaN           NaN
 
       id     status     work_type  has_data
300   301  Completed   data_worker       1.0
634   301  Completed       Manager       NaN
968   301  Completed  pursing_work       NaN
1302  301  Completed  not_in_field       NaN




## Long to Wide with `pivot`

Let's say we want to convert the data in the opposite way.  We have our `melted` dataframe in "long-form", but we want to create a new dataframe in "wide-form" (just like the dataframe we started with).

In [157]:
# re-add our 'has data' column
melted2 = melted.copy()
melted2['has_data'] = melted2.id.map(lambda x: 1)

pivoted = melted2.pivot(
    index='id',
    columns='work_type', 
    values='has_data').reset_index()

pivoted.columns.name = None # this is unecessary, but the column index name is confusing when displayed in jupyter

pivoted

,id,Manager,data_worker,not_in_field,pursing_work
0,1,NaN,1.0,NaN,NaN
1,2,NaN,1.0,NaN,NaN
2,3,NaN,1.0,NaN,NaN
3,4,NaN,1.0,NaN,NaN
4,5,NaN,1.0,NaN,NaN
...,...,...,...,...,...
327,330,NaN,1.0,NaN,NaN
328,331,NaN,NaN,NaN,1.0
329,332,NaN,NaN,NaN,1.0
330,333,NaN,1.0,NaN,NaN


In [1]:
df

NameError: name 'df' is not defined

In [2]:
ls = ["alexei","is","great"]


In [6]:
[a.capitalize() for a in ls]

['Alexei', 'Is', 'Great']

In [9]:
import pandas as pd
import numpy as np

data = pd.DataFrame({
    'age' :     [ 10, 22, 13, 21, 12, 11, 17],
    'section' : [ 'A', 'B', 'C', 'B', 'B', 'A', 'A'],
    'city' :    [ 'Gurgaon', 'Delhi', 'Mumbai', 'Delhi', 'Mumbai', 'Delhi', 'Mumbai'],
    'gender' :  [ 'M', 'F', 'F', 'M', 'M', 'M', 'F'],
    'favourite_color' : [ 'red', np.NAN, 'yellow', np.NAN, 'black', 'green', 'red']
})


In [10]:
data

,age,section,city,gender,favourite_color
0,10,A,Gurgaon,M,red
1,22,B,Delhi,F,NaN
2,13,C,Mumbai,F,yellow
3,21,B,Delhi,M,NaN
4,12,B,Mumbai,M,black
5,11,A,Delhi,M,green
6,17,A,Mumbai,F,red


In [12]:
data.loc[(data.age >= 12) & 
         (data.gender == 'M')]

,age,section,city,gender,favourite_color
3,21,B,Delhi,M,NaN
4,12,B,Mumbai,M,black


In [22]:
data.loc[[1],['age','city']]

,age,city
1,22,Delhi


In [35]:
data.iloc[1:3,0:3]

,age,section,city
1,22,B,Delhi
2,13,C,Mumbai
